In [1]:
import pandas as pd
import nltk

In [20]:
newzy = pd.read_csv('../FinalProject/newzy.csv', sep='|', index_col='doc_id')

/Users/matthewthomas/anaconda3/envs/DS5001/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
OHCO = ['doc_source','doc_id', 'sent_num', 'token_num']

In [4]:
newzy.groupby('doc_source').count()

,doc_title,doc_content,doc_date,doc_url
doc_source,,,,
Breitbart,10012,10011,10012,10012
CNN,38011,36184,38011,38011
Daily Kos,12310,12307,12310,12310
Drudge Report,77063,77063,77063,77063
Fox,83415,58077,83417,83417
Google News,64079,64079,64079,64078
Guardian,12392,12392,12392,12392
NPR,30755,24708,30755,30755
New York Times,21239,21145,21240,21240


## Organize and Tokenize PowerLine Articles

In [21]:
powerline = newzy[newzy.doc_source.isin(['PowerLine'])]

In [22]:
powerline = powerline.reset_index().set_index(OHCO[:2])
powerline.head(3)

doc_title  \
doc_source doc_id                                                 
PowerLine  48                             These Zins Are a Dusi   
           49      NY Times Editorial Board Hacked by The Onion   
           50                            The line Obama crossed   

                                                         doc_content  \
doc_source doc_id                                                      
PowerLine  48      (Steven Hayward) I’ve been falling behind on m...   
           49      (John Hinderaker) At least I think it was the ...   
           50      (Scott Johnson) Obamacare was enacted on a fou...   

                     doc_date  \
doc_source doc_id               
PowerLine  48      11/03/2013   
           49      11/03/2013   
           50      11/03/2013   

                                                             doc_url  
doc_source doc_id                                                     
PowerLine  48      http://feeds.powerlineblog.com/~r/powerlineblo...  
           49      http://feeds.powerlineblog.com/~r/powerlineblo...  
           50      http://feeds.powerlineblog.com/~r/powerlineblo...

In [6]:
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)

In [7]:
powerline.doc_content.str.len().describe()

count     9730.000000
mean      2844.561871
std       1942.034802
min         91.000000
25%       1532.250000
50%       2506.000000
75%       3692.000000
max      40421.000000
Name: doc_content, dtype: float64

In [8]:
#filter out very short articles
powerline = powerline[powerline['doc_content'].str.len() > 300]

In [9]:
len(powerline)

9664

In [10]:
#inspect articles to see what needs to be filtered
powerline.doc_content[:20]

doc_source  doc_id
PowerLine   48                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [11]:
#get rid of the jquery stuff
powerline['doc_content'] = powerline['doc_content'].str.replace(r'if\(typeof\(jQuery.*', '')

In [12]:
powerline.doc_content[:1]

doc_source  doc_id
PowerLine   48        (Steven Hayward) I’ve been falling behind on many regular features these days, among them, the Power Line 100, but also the crucial subject of wine.  So here’s a quick catch up guide. Old timers of Paso Robles wines all know the Dusi name.  The Dusi family’s Zinfandel vineyards go back nearly a century.  For decades, if you wanted to find the best Zinfandels from the region, you wanted to find who bought the Dusi crop for that year.  The Dusi family seldom made their own wine—until now.  And here’s the four-minute storyboard for it.  (Footnote: I’ve met Janell Dusi a couple of times, and always said, “When are you guys going to start keeping your own grapes for yourselves?”  The time has come.) 
Name: doc_content, dtype: object

In [13]:
#Take out authors in parantheses. This will delete some other words unfortunately
powerline['doc_content'] = powerline['doc_content'].str.replace(r'\([\w\s]*', '')
powerline.doc_content[:1]


doc_source  doc_id
PowerLine   48        ) I’ve been falling behind on many regular features these days, among them, the Power Line 100, but also the crucial subject of wine.  So here’s a quick catch up guide. Old timers of Paso Robles wines all know the Dusi name.  The Dusi family’s Zinfandel vineyards go back nearly a century.  For decades, if you wanted to find the best Zinfandels from the region, you wanted to find who bought the Dusi crop for that year.  The Dusi family seldom made their own wine—until now.  And here’s the four-minute storyboard for it.  : I’ve met Janell Dusi a couple of times, and always said, “When are you guys going to start keeping your own grapes for yourselves?”  The time has come.) 
Name: doc_content, dtype: object

In [14]:
#Split up sentences
df = powerline['doc_content'].str.split(r'[.?!;:"\(\)]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sentence'})

In [15]:
df = df[~df['sentence'].str.match(r'^\s*$')] # Remove empty sentences
df.head()

sentence
doc_source doc_id                                                                                                                                       
PowerLine  48     1   I’ve been falling behind on many regular features these days, among them, the Power Line 100, but also the crucial subject of wine
                  2                                                                                                     So here’s a quick catch up guide
                  3                                                                               Old timers of Paso Robles wines all know the Dusi name
                  4                                                                       The Dusi family’s Zinfandel vineyards go back nearly a century
                  5    For decades, if you wanted to find the best Zinfandels from the region, you wanted to find who bought the Dusi crop for that year

In [16]:
df.index.names = OHCO[:3]
df.head()

sentence
doc_source doc_id sent_num                                                                                                                                     
PowerLine  48     1          I’ve been falling behind on many regular features these days, among them, the Power Line 100, but also the crucial subject of wine
                  2                                                                                                            So here’s a quick catch up guide
                  3                                                                                      Old timers of Paso Robles wines all know the Dusi name
                  4                                                                              The Dusi family’s Zinfandel vineyards go back nearly a century
                  5           For decades, if you wanted to find the best Zinfandels from the region, you wanted to find who bought the Dusi crop for that year

In [17]:
#split into tokens
df = df['sentence'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame().rename(columns={0:'token'})

In [18]:
df.index.names = OHCO
df = df[~df['token'].str.match(r'^\s*$')]
df.head()

token
doc_source doc_id sent_num token_num         
PowerLine  48     1        1             I’ve
                           2             been
                           3          falling
                           4           behind
                           5               on

In [19]:
#from nltk.tag import StanfordPOSTagger
#stanford_pos_path = '../StanfordParser/stanford-postagger-2018-10-16/stanford-postagger.jar'

In [20]:
#st = StanfordPOSTagger('../StanfordParser/stanford-postagger-2018-10-16/models/english-bidirectional-distsim.tagger', stanford_pos_path)

In [21]:
def add_pos_to_tokens(tokens, ohco=OHCO[:3], token_col='token'):
    """Adds POS tags to tokens."""
    df = tokens.groupby(ohco).token.apply(lambda x: nltk.pos_tag(x.tolist()))\
        .apply(pd.Series).stack()\
        .to_frame().reset_index()\
        .rename(columns={'level_{}'.format(len(ohco)):'token_id', 0:'pos'})\
        .set_index(ohco + ['token_id'])
    tokens['pos'] = df.pos.apply(lambda x: x[1])
    return tokens 

In [22]:
add_pos_to_tokens(df)

token  pos
doc_source doc_id  sent_num token_num                 
PowerLine  48      1        1                I’ve  VBN
                            2                been  VBG
                            3             falling   IN
                            4              behind   IN
                            5                  on   JJ
...                                           ...  ...
           1026333 27       21             people  VBZ
                            22         represents   DT
                            23                  a   JJ
                            24                new   JJ
                            25                low  NaN

[4490019 rows x 2 columns]

## Tokenize Daily Kos Articles

In [16]:
kos = pd.read_csv('dailykos.csv', index_col = OHCO[:2])

In [17]:
del kos['Unnamed: 0']

In [18]:
kos = kos.drop('doc_content', axis=1).rename(columns={'articles':'doc_content'})
kos

doc_title  \
doc_source doc_id                                                      
Daily Kos  96      Obama has done nothing to address income inequ...   
           97      Anti-racism is not 'anti-white,' no matter wha...   
           98      Abbreviated Pundit Round-up:     Angry does no...   
           99                         Sunday Talk: Haters gonna hate   
           100     Gore on fire: Says humans treating atmosphere ...   
...                                                              ...   
           922223  CNN's new Republican hack is ratioed trying to...   
           922224  California announces investigation into PG&amp...   
           922225  Chicago teachers say 0.5% of the schools budge...   
           922226  How do we engage people who've been sitting on...   
           922227  Formerly incarcerated leaders in criminal just...   

                     doc_date  \
doc_source doc_id               
Daily Kos  96      11/03/2013   
           97      11/03/2013   
           98      11/03/2013   
           99      11/03/2013   
           100     11/03/2013   
...                       ...   
           922223  10/31/2019   
           922224  10/31/2019   
           922225  10/31/2019   
           922226  10/31/2019   
           922227  10/31/2019   

                                                             doc_url  \
doc_source doc_id                                                      
Daily Kos  96      http://feeds.dailykos.com/~r/dailykos/index/~3...   
           97      http://feeds.dailykos.com/~r/dailykos/index/~3...   
           98      http://feeds.dailykos.com/~r/dailykos/index/~3...   
           99      http://feeds.dailykos.com/~r/dailykos/index/~3...   
           100     http://feeds.dailykos.com/~r/dailykos/index/~3...   
...                                                              ...   
           922223  http://feeds.dailykosmedia.com/~r/dailykos/ind...   
           922224  http://feeds.dailykosmedia.com/~r/dailykos/ind...   
           922225  http://feeds.dailykosmedia.com/~r/dailykos/ind...   
           922226  http://feeds.dailykosmedia.com/~r/dailykos/ind...   
           922227  http://feeds.dailykosmedia.com/~r/dailykos/ind...   

                                                         doc_content  
doc_source doc_id                                                     
Daily Kos  96      But how strong is the correlation between fall...  
           97      The virulent meme spread across our nation wit...  
           98      Why are the plutocrats, with their great wealt...  
           99      Unfortunately for Barack Obama, a growing numb...  
           100     Al Gore has been everywhere lately. Just last ...  
...                                                              ...  
           922223  Earlier today, former Republican Rep. from Wis...  
           922224  On Monday, the California Public Utilities Com...  
           922225  Chicago teachers say that just half of one per...  
           922226  Ever wondered how, precisely, social movements...  
           922227  On Monday, three candidates running for the De...  

[12310 rows x 4 columns]

In [31]:
#inspect articles to clean them up
kos.doc_content.head(10)

doc_source  doc_id
Daily Kos   96                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [32]:
#Get rid of short articles and bad links
df_kos = kos[kos.articles.str.len() > 500]

In [33]:
df_kos = df_kos['doc_content'].str.split(r'[.?!;:"\(\)]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sentence'})

df_kos = df_kos[~df_kos['sentence'].str.match(r'^\s*$')]

df_kos = df_kos['sentence'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame().rename(columns={0:'token'})

df_kos.index.names = OHCO
df_kos = df_kos[~df_kos['token'].str.match(r'^\s*$')]
df_kos.head()


token
doc_source doc_id sent_num token_num        
Daily Kos  96     0        0             But
                           1             how
                           2          strong
                           3              is
                           4             the

In [34]:
add_pos_to_tokens(df_kos)

token  pos
doc_source doc_id sent_num token_num              
Daily Kos  96     0        0              But   CC
                           1              how  WRB
                           2           strong   JJ
                           3               is  VBZ
                           4              the   DT
...                                       ...  ...
           922227 43       15             one   NN
                           16             way   CC
                           17              or   DT
                           18         another  NaN
                  44       0                ”   NN

[7961908 rows x 2 columns]

## Tokenize Politico

In [11]:
politico = pd.read_csv('politico.csv', index_col=OHCO[:2])

In [12]:
del politico['Unnamed: 0']

In [37]:
politico.doc_content.head(10)

doc_source         doc_id
Politico Magazine  1542            On Jan. 1, 1943, Robert Gene Baker arrived in Washington at the height of World War II to become a Senate page. Two decades later, this son of a mailman from Pickens, S.C., had become the reigning Washington wheeler-dealer and fixer of his day as secretary to the Senate’s Democratic majoritContinue reading ...
                   1545                          This week, world powers and Iran may well be poised to reach an historic agreement to limit the Islamic Republic’s nuclear program—a diplomatic feat that promises to reorder the Middle East and neutralize a threat that has loomed over the region for a decade. The talks, held iContinue reading ...
                   1547                                President Barack Obama has gamely acknowledged that the HealthCare.gov disaster is “on me,” and surely this is one of the more accurate assertions he’s made related to his signature health care program. But no president could

In [38]:
df_politico = politico[politico.doc_content.str.len() > 500]

In [39]:
df_politico = df_politico['doc_content'].str.split(r'[.?!;:"\(\)]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sentence'})

df_politico = df_politico[~politico['sentence'].str.match(r'^\s*$')]

df_politico = df_politico['sentence'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame().rename(columns={0:'token'})

df_politico.index.names = OHCO
df_politico = df_politico[~df_politico['token'].str.match(r'^\s*$')]
df_politico.head()

token
doc_source        doc_id  sent_num token_num           
Politico Magazine 1010531 0        0                The
                                   1          candidate
                                   2              moves
                                   3             slowly
                                   4            inching

In [40]:
add_pos_to_tokens(df_politico)

token  pos
doc_source        doc_id  sent_num token_num                
Politico Magazine 1010531 0        0                The   DT
                                   1          candidate   NN
                                   2              moves  VBZ
                                   3             slowly   RB
                                   4            inching  VBG
...                                                 ...  ...
                  1009593 206      4                the  NNS
                                   5              cards   IN
                                   6                 at   DT
                                   7                all  NaN
                          207      0                  ”   NN

[7287455 rows x 2 columns]

In [23]:
LIBRARY = pd.concat([powerline, kos, politico])
LIBRARY

doc_title  \
doc_source        doc_id                                                       
PowerLine         48                                   These Zins Are a Dusi   
                  49            NY Times Editorial Board Hacked by The Onion   
                  50                                  The line Obama crossed   
                  51                                                Sham/Wow   
                  52       A Quarterback Controversy With a Political Twist?   
...                                                                      ...   
Politico Magazine 1009144  Opinion | Impeachment Was Supposed to Protect ...   
                  1009145  What Will the History Books Say About This Imp...   
                  1009454      Democracy in Inaction: How Trump Beat the Rap   
                  1009592  What If Everything You Think You Know About Po...   
                  1009593                      The Trump Lawyer Who Survived   

                                                                 doc_content  \
doc_source        doc_id                                                       
PowerLine         48       (Steven Hayward) I’ve been falling behind on m...   
                  49       (John Hinderaker) At least I think it was the ...   
                  50       (Scott Johnson) Obamacare was enacted on a fou...   
                  51       (Scott Johnson) It’s unfair to compare and con...   
                  52       (John Hinderaker) Public Policy Polling ran a ...   
...                                                                      ...   
Politico Magazine 1009144  Lawmakers from both parties have sown doubt th...   
                  1009145  History Dept. We knew how this would end. We a...   
                  1009454  Our political culture has allowed its muscles ...   
                  1009592  Rachel Bitecofer’s radical new theory predicte...   
                  1009593  The most theatrical lawyer in the impeachment ...   

                             doc_date  \
doc_source        doc_id                
PowerLine         48       11/03/2013   
                  49       11/03/2013   
                  50       11/03/2013   
                  51       11/03/2013   
                  52       11/03/2013   
...                               ...   
Politico Magazine 1009144  02/05/2020   
                  1009145  02/05/2020   
                  1009454  02/06/2020   
                  1009592  02/06/2020   
                  1009593  02/06/2020   

                                                                     doc_url  
doc_source        doc_id                                                      
PowerLine         48       http://feeds.powerlineblog.com/~r/powerlineblo...  
                  49       http://feeds.powerlineblog.com/~r/powerlineblo...  
                  50       http://feeds.powerlineblog.com/~r/powerlineblo...  
                  51       http://feeds.powerlineblog.com/~r/powerlineblo...  
                  52       http://feeds.powerlineblog.com/~r/powerlineblo...  
...                                                                      ...  
Politico Magazine 1009144  https://www.politico.com/news/magazine/2020/02...  
                  1009145  https://www.politico.com/news/magazine/2020/02...  
                  1009454  https://www.politico.com/news/magazine/2020/02...  
                  1009592  https://www.politico.com/news/magazine/2020/02...  
                  1009593  https://www.politico.com/news/magazine/2020/02...  

[25364 rows x 4 columns]

In [40]:
TOKEN = pd.concat([df, df_kos, df_politico])

In [41]:
TOKEN['term_str'] = TOKEN['token'].str.lower().str.replace('[\W_]', '')

In [45]:
VOCAB = TOKEN.term_str.value_counts().to_frame().rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [46]:
VOCAB.head()

,term_str,n
term_id,,
0,,165366
1,0,1152
2,00,408
3,000,10643
4,0000,1


### Add Stopwords

In [47]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [48]:
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

### Add Stems

In [49]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
VOCAB['p_stem'] = VOCAB.term_str.apply(stemmer.stem)

### Add Term ID to Token Table

In [51]:
TOKEN['term_id'] = TOKEN.term_str.map(VOCAB.reset_index().set_index('term_str').term_id)

In [53]:
TOKEN.head()

token  pos term_str  term_id
doc_source doc_id sent_num token_num                                
PowerLine  48     1        1             I’ve  VBN      ive    85795
                           2             been  VBG     been    17768
                           3          falling   IN  falling    61167
                           4           behind   IN   behind    18021
                           5               on   JJ       on   114185

### Add POS max to Vocab Table

In [52]:
VOCAB['pos_max'] = TOKEN.groupby(['term_id', 'pos']).pos.count().unstack().idxmax(1)

In [54]:
VOCAB.head()

,term_str,n,stop,p_stem,pos_max
term_id,,,,,
0,,165366,0,,NN
1,0,1152,0,0,CD
2,00,408,0,00,CD
3,000,10643,0,000,NNS
4,0000,1,0,0000,CD


### Add Term Rank to Vocab

In [55]:
if 'term_rank' not in VOCAB.columns:
    VOCAB = VOCAB.sort_values('n', ascending=False).reset_index()
    VOCAB.index.name = 'term_rank'
    VOCAB = VOCAB.reset_index()
    VOCAB = VOCAB.set_index('term_id')
    VOCAB['term_rank'] = VOCAB['term_rank'] + 1

In [56]:
VOCAB.head()

,term_rank,term_str,n,stop,p_stem,pos_max
term_id,,,,,,
158064,1,the,1103705,1,the,NN
160412,2,to,558006,1,to,VB
113256,3,of,509160,1,of,DT
10479,4,and,451366,1,and,NNP
4833,5,a,439586,1,a,NN


In [57]:
TOKEN.to_csv('TOKEN.csv')

In [58]:
VOCAB.to_csv('VOCAB.csv')

In [25]:
LIBRARY.to_csv('LIBRARY.csv')